In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
from tqdm import tqdm
import yfinance as yf

In [2]:
DIR_DATA = '/Users/mohanli/Documents/citadel/46924_Natual_Language_Processing-Applications_in_Finance/proj1/data/'
df_raw = pd.read_excel(
    DIR_DATA + 'sp500_w_addl_id_with_cik.xlsx', index_col=0)


In [3]:
df_raw

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret,cik,sic,naics,gind,gsubind
0,2011-01-31,60986,NEWELL RUBBERMAID INC,65122910,11,1,3089,NWL,7875,1,1989-04-27,2022-03-31,0.058856,814453.0,3990,3399,252010,25201050
1,2011-01-31,85914,BEST BUY COMPANY INC,8651610,11,1,5731,BBY,2184,1,1999-06-30,2022-03-31,-0.008457,764478.0,5731,443142,255040,25504020
2,2011-01-31,80711,APARTMENT INVESTMENT & MGMT CO,03748R10,18,1,6798,AIV,30490,1,2003-03-14,2020-12-18,-0.010836,922864.0,6798,531110,601010,60101060
3,2011-01-31,59176,AMERICAN EXPRESS CO,2581610,11,1,6141,AXP,1447,1,1976-07-01,2022-03-31,0.014912,4962.0,6141,522210,402020,40202010
4,2011-01-31,75100,TIFFANY & CO NEW,88654710,11,1,5944,TIF,13646,1,2000-06-21,2021-01-06,-0.066485,98246.0,5944,448310,255040,25504040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66113,2021-12-31,83143,IRON MOUNTAIN INC NEW,46284V10,18,1,4226,IRM,62374,1,2009-01-06,2022-03-31,0.165240,1020569.0,6798,531120,601010,60101080
66114,2021-12-31,76605,AUTOZONE INC,5333210,11,1,5531,AZO,23809,1,1997-01-02,2022-03-31,0.153720,866787.0,5531,441310,255040,25504050
66115,2021-12-31,60097,MEDTRONIC PLC,G5960L10,12,1,3845,MDT,7228,1,1986-10-23,2022-03-31,-0.024555,1613103.0,3845,334510,351010,35101010
66116,2021-12-31,27828,H P INC,40434L10,11,1,3571,HPQ,5606,1,1974-10-17,2022-03-31,0.074830,47217.0,3570,33411,452020,45202030


In [4]:
# select ones present in sp500 after 2016
df = df_raw[df_raw['ending'] > dt.datetime(2015, 12, 31)].copy()

In [ ]:
# the real download!
for ticker in tqdm(df['ticker'].unique()):
    file_name = DIR_DATA + f'ret/{ticker}.pkl'
    if os.path.exists(file_name):
        continue
    data = yf.download(
        tickers=ticker,
        start="2016-01-01",
        end="2021-12-31",
        interval="1d",
    )
    if len(data) == 0:
        continue
    data['symbol'] = ticker
    data = data.drop(['High', 'Low', 'Volume'], axis=1)
    data.to_pickle(file_name)
    # break


In [5]:
ticker_ls = []
for file_ in os.listdir(DIR_DATA + 'ret/'):
    if file_[-1] == 'l':
        tmp = pd.read_pickle(DIR_DATA + 'ret/' + file_)
        # if len(tmp) != 0:
        ticker_ls.append(file_.split('.')[0])

In [6]:
len(ticker_ls)

584

In [7]:
df_new = df[df['ticker'].isin(ticker_ls)]

In [9]:
tmp = []
for ticker in df_new['ticker'].unique():
    if len(df_new.loc[df_new['ticker'] == ticker, 'cik'].unique()) > 1:
        tmp.append(ticker)
len(tmp)

8

In [10]:
tmp = []
for cik in df_new['cik'].unique():
    if len(df_new.loc[df_new['cik'] == cik, 'ticker'].unique()) > 1:
        tmp.append(cik)
len(tmp)

13

In [14]:
df_new.to_csv(DIR_DATA + 'filtered_sp500.csv')